In [ ]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.model_selection import (train_test_split, cross_val_score,
                                     cross_val_predict, GridSearchCV, RandomizedSearchCV)

from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from collections import Counter
from sklearn.model_selection import TimeSeriesSplit
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

# 1.0 Data Preprocessing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import talib as ta
%matplotlib inline

In [ ]:
# download 30 years of daily data from Yahoo Finance

tickers = ['SPY']
df = yf.download(" ".join(tickers), period='30y', interval='1d', group_by='tickers')

# 1.1 Define predictor variables and a target variable

In [ ]:
df.reset_index(inplace=True)

In [ ]:
# Time features
df['year'] = df.loc[:, 'Date'].dt.year
df['month'] = df.loc[:, 'Date'].dt.month
df['day_of_week'] = df.loc[:, 'Date'].dt.day_of_week
df['week'] = df.loc[:, 'Date'].dt.isocalendar().week

In [ ]:
tmp = df.copy()
tmp.set_index("Date", inplace=True)
tmp = tmp.loc[:,'Adj Close'].rolling(7).mean()

In [ ]:
import datetime as dt

fig, axl = plt.subplots()
fig.set_size_inches(10,7)
plt.rcParams['lines.linewidth'] = 2
axl.plot(tmp.dropna().index, tmp.dropna(), label='Price', color='black')
axl.vlines(dt.datetime(2003,1,30), ymin=0, ymax=400, color='blue')
axl.vlines(dt.datetime(2009,1,30), ymin=0, ymax=400, color='blue')
axl.vlines(dt.datetime(2020,2,29), ymin=0, ymax=400, color='blue')
plt.ylabel('Price')
plt.xlabel('Date')
plt.grid(False)
plt.show()

In [ ]:
df

## 1.1.1 Feature engineering

In [ ]:
# High - Low
df['H-L'] = df.High - df.Low
# Open - Close
df['O-C'] = df.Open - df.Close
# Correlation
df['volume_by_adv20'] = df.Volume/df.Volume.rolling(20).mean()

# Pass previous High, Low, Close for the alogoritm to have a sense of volatility in the past
df['Prev_High'] = df['High'].shift(1)
df['Prev_Low'] = df['Low'].shift(1)
df['Prev_Close'] = df['Close'].shift(1)

# Create columns 'OO' with the difference between the current minute's open and last day's open
df['OO'] = df['Open']-df['Open'].shift(1)

# Create columns 'OC' with the difference between the current minute's open and prevo
df['OC'] = df['Open']-df['Prev_Close']


In [ ]:
################# TECHNICAL INDICATORS ######################
def get_momentum(prices, window):
    momentum = prices / prices.shift(window) - 1
    return momentum

def get_bb(prices, window):
    rm = prices.rolling(window).mean()
    rstd = prices.rolling(window).std()
    bbp = (prices - rm) / 2 * rstd
    return bbp 

def get_psma(prices, window):
    rm = prices.rolling(window).mean()
    psma = prices.divide(rm, axis=0) - 1
    return psma 

def get_pema(prices, window):
    ema = prices.ewm(window).mean()
    pema = prices.divide(ema, axis=0) - 1
    return pema


# Create a lookback period(n) = 10-days
n=10

# 1. Relative Strength Index (RSI)
df['RSI'] = ta.RSI(df['Adj Close'].shift(-1), timeperiod=n)

# 2. SMA
df['SMA'] = df['Adj Close'].shift(1).rolling(window=n).mean()

# 3. Correlation between Adj Close and SMA
df['Corr'] = df['Adj Close'].shift(1).rolling(window=n).corr(df.SMA.shift(1))

# 4. Parabolic SAR (stop and reverse)
df['SAR'] = ta.SAR(df.High.shift(1), df.Low.shift(1), 0.2, 0.2)

# 5. ADX (Average directional movement index)
df['ADX'] = ta.ADX(df.High.shift(1), df.Low.shift(1), df.Open, timeperiod=n)

# 6. NATR (Normalized average true range)
df['NATR'] = ta.NATR(df.Low,df.High,df.Close, timeperiod=n)

# 7. Bollinger bands
df['BB'] = get_bb(df['Adj Close'], window=n)

# 8. Price / SMA
df['PSMA'] = get_psma(df['Adj Close'], window=n)

# 9. Price / EMA
df['PEMA'] = get_pema(df['Adj Close'], window=n)

# 10. Momentum
df['MOM'] = get_momentum(df['Adj Close'], window=n)
df['MOM10'] = get_momentum(df['Adj Close'], window=20)
df['MOM30'] = get_momentum(df['Adj Close'], window=30)
df['MOM40'] = get_momentum(df['Adj Close'], window=40)

In [ ]:
# returns
df['ret1'] = df['Adj Close'].pct_change()

df['ret5'] = df['ret1'].rolling(5).sum()
df['ret10'] = df['ret1'].rolling(10).sum()
df['ret20'] = df['ret1'].rolling(20).sum()
df['ret40'] = df['ret1'].rolling(40).sum()

# One-day future returns
df['retFut1'] = df.ret1.shift(-1)

# calculate past returns to help algorithm understand the trends in the last n periods
for i in range(1, n):
    df['return%i' % i] = df.retFut1.shift(i)

In [ ]:
# Change the value of 'Corr' to -1 if it is less than -1
df.loc[df.Corr < -1, 'Corr'] = -1

# Change the value of 'Corr' to 1 if it is greater than 1
df.loc[df['Corr'] > 1, 'Corr'] = 1

# Drop the NaN values
df = df.dropna()

# 1.1.2 Train and test data

In [ ]:
t = .8
split = int(t*len(df))

# 1.1.3 Create output signals

1. Hightest returns' quantile is assigned Signal 1 (or Buy)
2. Middle quantile is assigned Signal 0 (or do nothing)
3. Lowest quantile is assigned Signal -1 (or Sell)

In [ ]:
df['Signal'] = 0
df.loc[df.retFut1 > df.retFut1[:split].quantile(q=0.66), 'Signal'] = 1
df.loc[df.retFut1 < df.retFut1[:split].quantile(q=0.34), 'Signal'] = -1

### Data Summary

In [ ]:
from pandas_profiling import ProfileReport

data = df.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume','retFut1',
       'return1', 'return2', 'return3', 'return4', 'return5', 'return6',
       'return7', 'return8', 'return9'], axis=1)

ProfileReport(data)

In [ ]:
X = df.drop(['Date','Open','Close','Adj Close','Signal', 'High','Low', 'Volume', 'retFut1'], axis=1)
y = df['Signal']

In [ ]:
# multiclass classification
Counter(y)

# 1. DECISION TREE CLASSIFIER

## Cost complexity pruning

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
X_train, X_test,y_train, y_test = X[:split], X[split:], y[:split], y[split:]
clf = DecisionTreeClassifier(random_state=42)
path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle='steps-post')
ax.set_xlabel('effective alpha')
ax.set_ylabel('total impurity of leaves')
ax.set_title("Total Impuritiy vs. effective Alpha for training set")

In [ ]:
# train the decision tree using the effective alphas
clfs = []

for aa in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=42, ccp_alpha=aa)
    clf.fit(X_train, y_train)
    clfs.append(clf)
    
print(
    f"Number of nodes in the last tree is: {clfs[-1].tree_.node_count} with ccp_alpha: {ccp_alphas[-1]}")

In [ ]:
# show how the number of tree depth and nodes decrease as alpha increases

clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]

node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]

fig, ax = plt.subplots(figsize=(8,6), nrows=2, ncols=1)
ax[0].plot(ccp_alphas, node_counts, marker='o', drawstyle='steps-post')
ax[0].set_xlabel('alpha')
ax[0].set_ylabel('number of nodes')
ax[0].set_title('Number of nodes vs alpha')
ax[1].plot(ccp_alphas, depth, marker="o", drawstyle="steps-post")
ax[1].set_xlabel("alpha")
ax[1].set_ylabel("depth of tree")
ax[1].set_title("Depth vs alpha")
fig.tight_layout()

In [ ]:
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

fig, ax = plt.subplots(figsize=(8,6))
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker="o", label="train", drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker="o", label="test", drawstyle="steps-post")
ax.legend()
plt.show()

# Baseline Decision Tree

This DT is un-pruned. This will act as a baseline for more improved versions of the tree

In [ ]:
X_train, X_test,y_train, y_test = X[:split], X[split:], y[:split], y[split:]

baseline_model = DecisionTreeClassifier(random_state=42)
baseline_model.fit(X_train, y_train)
y_pred = baseline_model.predict(X_test)
print(classification_report(y_test, y_pred))

# Hyper parameter Tuning

1.resample the training set to oversample minority class

The key hyperparameters in a DTree are
* max_leaf_node,
* max_features,
* max_depth,
* min_samples_leaf,
* class_weight,
* and ccp_alpha

In [ ]:
Counter(y_train)

In [ ]:
# Number of features to consider at every split
max_features = [round(i, 2) for i in np.linspace(start=0.3, stop=1.0, num=5)]

# Max depth of the tree
max_depth = [round(x, 2) for x in np.linspace(start=2, stop=20, num=5)]

# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in np.linspace(start=50, stop=600, num=20)]

# Maximum leaves at each node
max_leaf_nodes = [int(x) for x in np.linspace(start=100, stop=1000, num=20)]

# Cost complexity penalty (pruning)
ccp_alpha = [round(x, 4) for x in np.linspace(start=0.001, stop=0.05, num=100)]

param_grid = {'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_leaf': min_samples_leaf,
              'max_leaf_nodes': max_leaf_nodes,
              'ccp_alpha': ccp_alpha
              }

In [ ]:
dtree = DecisionTreeClassifier()
cv = TimeSeriesSplit(n_splits=5)
dt_rcv = RandomizedSearchCV(estimator=dtree,
                              param_distributions=param_grid,
                              n_iter=50,
                              random_state=42,
                              cv=cv,
                              n_jobs=8
                              )

# fit model
dt_rcv.fit(X_train, y_train)
# extract best params
best_params = dt_rcv.best_params_
best_params

# Validation Curve

In [ ]:
from sklearn.model_selection import validation_curve, learning_curve
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.metrics import roc_auc_score

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
val_curve = ValidationCurve(DecisionTreeClassifier(),
                            param_name='max_depth',
                           param_range=max_depth,
                           scoring='accuracy',
                           n_jobs=-1,
                            cv=TimeSeriesSplit(n_splits=5),
                           ax=ax)

val_curve.fit(X_train, y_train)
val_curve.poof()
sns.despine()
fig.tight_layout();

# Learning Curve

In [ ]:
dt_rcv.best_estimator_

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
(pd.Series(dt_rcv.best_estimator_.feature_importances_, 
           index=X.columns)
 .sort_values(ascending=False)
 .iloc[:5]
 .sort_values()
 .plot.barh(ax=ax, title='DT Feature Importance'))
sns.despine()
fig.tight_layout();

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
learn_curve = LearningCurve(dt_rcv.best_estimator_,
                           train_sizes=np.arange(.1, 1.01, .1),
                           scoring='accuracy',
                           cv=TimeSeriesSplit(n_splits=5),
                           ax=ax)

learn_curve.fit(X_train, y_train)
learn_curve.poof()
sns.despine()
fig.tight_layout();

# Final model prediction

In [ ]:
best_params

In [ ]:
model = DecisionTreeClassifier(max_depth=20.0,
                              max_features=1.0,
                              max_leaf_nodes=1000,
                              min_samples_leaf=78,
                              ccp_alpha=.0069)
%time model.fit(X_train, y_train)
%time y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# K-fold Cross Validation (Generalization Error)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv_score = cross_val_score(estimator=model,
                          X=X_train,
                          y=y_train,
                          cv=TimeSeriesSplit(n_splits=10),
                          n_jobs=-1,
                          verbose=1)

In [ ]:
cv_score

In [ ]:
"Accuracy: %.3f%% (%.3f%%)" % (cv_score.mean()*100.0, cv_score.std()*100.0)

# Performance Metrics

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import roc_auc_score

In [ ]:
kf = KFold(n_splits=10, shuffle=False)
y_score = cross_val_predict(model, X=X, y=y, cv=kf.split(X),method='predict_proba')

pred_scores = dict(y_true=y, y_score=y_score)

# ROC AUC
roc_auc_score(**pred_scores, multi_class='ovr')

# 2. DECISION TREE CLASSIFIER WITH adaBOOST

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
best_params

## Hyperparameter tuning

In [ ]:
# Grid Search AdaBoost
param_grid = {'n_estimators': [int(x)for x in np.linspace(1, 50, 10)]}

ada = AdaBoostClassifier(
                base_estimator = DecisionTreeClassifier(
                            ccp_alpha=0.0069,
                            max_depth=20,
                            max_features=1.0,
                            max_leaf_nodes=1000,
                            min_samples_leaf=78),
                n_estimators = 50,
                random_state = 42
            )
cv = TimeSeriesSplit(n_splits=5)

ada_rcv = GridSearchCV(estimator=ada,
                              param_grid=param_grid,
                              cv=cv,
                              n_jobs=8
                              )

# fit model
ada_rcv.fit(X_train, y_train)
# extract best params
best_params = ada_rcv.best_params_
best_params

# Validation Curve

In [ ]:
n_estimators = [int(x) for x in np.linspace(start=1, stop=50, num=10)]

fig, ax = plt.subplots(figsize=(8, 6))
val_curve_2 = ValidationCurve(ada,
                      param_name='n_estimators',
                      param_range=n_estimators,
                      cv=TimeSeriesSplit(n_splits=5),
                      scoring='accuracy',
                      n_jobs=-1,
                      ax=ax)
val_curve_2.fit(X_train, y_train)
val_curve_2.poof()
sns.despine()
fig.tight_layout();

# Learning Curve

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
learn_curve = LearningCurve(ada,
                           train_sizes=np.arange(.1, 1.01, .1),
                           scoring='accuracy',
                           cv=TimeSeriesSplit(n_splits=5),
                           ax=ax)

learn_curve.fit(X_train, y_train)
learn_curve.poof()
sns.despine()
fig.tight_layout();

# Final model prediction

In [ ]:
# Define the AdaBoost model
ada_boost = AdaBoostClassifier(
                base_estimator = DecisionTreeClassifier(
                            ccp_alpha=0.0069,
                            max_depth=20,
                            max_features=1.0,
                            max_leaf_nodes=1000,
                            min_samples_leaf=78),
                n_estimators = 6,
                random_state = 42
            )

%time ada_boost.fit(X_train, y_train)

In [ ]:
%time y_pred = ada_boost.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
#  DTree tune model results
                precision    recall  f1-score   support

          -1       0.79      0.66      0.72       400
           0       0.57      0.59      0.58       595
           1       0.58      0.63      0.60       458

    accuracy                           0.62      1453
   macro avg       0.65      0.63      0.63      1453
weighted avg       0.63      0.62      0.63      1453


In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
(pd.Series(ada_boost.feature_importances_, 
           index=X.columns)
 .sort_values(ascending=False)
 .iloc[:10]
 .sort_values()
 .plot.barh(ax=ax, title='adaBOOST DT Feature Importance'))
sns.despine()
fig.tight_layout();

# K-fold Cross Validation (Generalization Error)

In [ ]:
cv_score = cross_val_score(estimator=ada_boost,
                          X=X_train,
                          y=y_train,
                          cv=TimeSeriesSplit(n_splits=10),
                          n_jobs=-1,
                          verbose=1)

print(cv_score)
print('-'*20)
"Accuracy: %.3f%% (%.3f%%)" % (cv_score.mean()*100.0, cv_score.std()*100.0)

# Performance metrics

In [ ]:
kf = KFold(n_splits=10, shuffle=False)
y_score = cross_val_predict(model, X=X, y=y, cv=kf.split(X),method='predict_proba')

pred_scores = dict(y_true=y, y_score=y_score)

# ROC AUC
roc_auc_score(**pred_scores, multi_class='ovr')

# 3. k-NN CLASSIFIER

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.neighbors import KNeighborsClassifier
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

# Baseline Model - KNN with L1-Manhattan (not suitable for time series)

In [ ]:
# Baseline Models (exploration)
tscv = TimeSeriesSplit(n_splits=5)
pipe = Pipeline([('scaler', StandardScaler()), 
                 ('knn', KNeighborsClassifier())])

param_grid = {
    'knn__metric': ['minkowski', 'euclidean','manhattan'],
    'knn__n_neighbors': tuple(range(5, 101, 10)),
    'knn__weights': ['uniform','distance'],
    'knn__p': [1,2]
}

estimator_knn = GridSearchCV(estimator=pipe,
                        param_grid=param_grid,
                        cv=tscv,
                        scoring='roc_auc',
                        n_jobs=-1)

%time estimator_knn.fit(X=X_train, y=y_train)

In [ ]:
estimator_knn.best_params_ # uses L1 (manhattan distance)

In [ ]:
knn_model = estimator_knn.best_estimator_

%time knn_model.fit(X_train, y_train)
%time y_pred = knn_model.predict(X_test)
print(classification_report(y_test, y_pred))

# k-NN DTW (suitable for time series classification)

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
pipe = Pipeline([('scaler', StandardScaler()), 
                 ('knn', KNeighborsTimeSeriesClassifier())])

param_grid = {
    'knn__metric': ['dtw', 'ctw','softdtw'],
    'knn__n_neighbors': tuple(range(5, 101, 10)),
    'knn__weights': ['uniform','distance']
}

estimator_knn = GridSearchCV(estimator=pipe,
                        param_grid=param_grid,
                        cv=tscv,
                        scoring='roc_auc',
                        n_jobs=-1)

estimator_knn.fit(X=X_train, y=y_train)

In [ ]:
estimator_knn.best_params_ # uses dtw

In [ ]:
knn_dtw = KNeighborsTimeSeriesClassifier(n_neighbors=5,
                                     weights='uniform',
                                    metric='dtw',
                                    n_jobs=-1)

In [ ]:
ss = StandardScaler().fit(X_train)

%time knn_dtw.fit(ss.fit_transform(X_train), y_train)

%time y_pred=knn_dtw.predict(ss.fit_transform(X_test))
print(classification_report(y_test, y_pred))

# Validation Curve

In [ ]:
from yellowbrick.model_selection import ValidationCurve, LearningCurve
from sklearn.model_selection import validation_curve, learning_curve
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.metrics import roc_auc_score

In [ ]:
estimator_knn.best_params_ # use knn (not suitable for time series for now)

In [ ]:
n_neighbors = tuple(range(5, 101, 10))
tscv = TimeSeriesSplit(n_splits=5)

fig, ax = plt.subplots(figsize=(8, 6))
val_curve = ValidationCurve(KNeighborsClassifier(metric='minkowski',
                                                 p=1, weights='uniform'),
                      param_name='n_neighbors',
                      param_range=n_neighbors,
                      cv=tscv,
                      scoring='accuracy',
                      n_jobs=-1,
                      ax=ax)
val_curve.fit(X_train, y_train)
val_curve.poof()
sns.despine()
fig.tight_layout();

# Learning Curve

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
learn_curve = LearningCurve(KNeighborsClassifier(),
                           train_sizes=np.arange(.1, 1.01, .1),
                           scoring='accuracy',
                           cv=TimeSeriesSplit(n_splits=5),
                           ax=ax)

learn_curve.fit(X_train, y_train)
learn_curve.poof()
sns.despine()
fig.tight_layout();

# K-fold Cross Validation (Generalization Error)

In [ ]:
cv_score = cross_val_score(estimator=estimator_knn.best_estimator_,
                          X=X_train,
                          y=y_train,
                          cv=TimeSeriesSplit(n_splits=10),
                          n_jobs=-1,
                          verbose=1)

print(cv_score)
print('-'*20)
"Accuracy: %.3f%% (%.3f%%)" % (cv_score.mean()*100.0, cv_score.std()*100.0)

# Performance metrics

In [ ]:
kf = KFold(n_splits=10, shuffle=False)
y_score = cross_val_predict(estimator_knn.best_estimator_,
                            X=X, y=y, cv=kf.split(X),method='predict_proba')

pred_scores = dict(y_true=y, y_score=y_score)

# ROC AUC
roc_auc_score(**pred_scores, multi_class='ovr')

# 4. SVM

# Hyperparameter tuning

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
steps = [('scaler', StandardScaler()), ('svc', SVC())]
pipeline = Pipeline(steps)

### Gaussian Kernel

In [ ]:
# test the different input for C and gamma
# C <-- the regularization parameter (trade-off between error and decision boundary)
# gamma <-- determines how far the influence of a single training example reaches 
C = [1e1, 1e2, 1e3, 1e4, 1e5]
gamma = [1e-3, 1e-2, 1e-1, 1e0]

# Initialize parameters
parameters = {'svc__C': C,
             'svc__gamma': gamma,
             'svc__kernel': ['rbf']
             }

rbf_rcv = RandomizedSearchCV(estimator=pipeline,
                        param_distributions=parameters,
                        cv=tscv,
                        n_jobs=-1)

# Training on and fetching the best parameters
rbf_rcv.fit(X_train, y_train)

In [ ]:
rbf_rcv.best_estimator_

### Polynomial Kernel

In [ ]:
C = [1e2, 1e3, 1e4, 1e5,1e6]
gamma = [1e-3, 1e-2, 1e-1, 1e0, 0.0]

# Initialize parameters
parameters = {'svc__C': C,
             'svc__gamma': gamma,
             'svc__kernel': ['poly']
             }

poly_rcv = RandomizedSearchCV(estimator=pipeline,
                        param_distributions=parameters,
                        cv=tscv,
                        n_jobs=-1)

# Training on and fetching the best parameters
poly_rcv.fit(X_train, y_train)

In [ ]:
poly_rcv.best_estimator_

### Sigmoid kernel

In [ ]:
C = [1e2, 1e3, 1e4, 1e5,1e6]
gamma = [1e-3, 1e-2, 1e-1, 1e0, 0.0]

# Initialize parameters
parameters = {'svc__C': C,
             'svc__gamma': gamma,
             'svc__kernel': ['sigmoid']
             }

sig_rcv = RandomizedSearchCV(estimator=pipeline,
                        param_distributions=parameters,
                        cv=tscv,
                        n_jobs=-1)

# Training on and fetching the best parameters
sig_rcv.fit(X_train, y_train)

In [ ]:
sig_rcv.best_estimator_

# Final model prediction

### Gaussian kernel

In [ ]:
# create SVC with the optimized hyperparameters from gridsearch
svc_rbf = rbf_rcv.best_estimator_

# Pass the scaled train data to the SVC classifier
%time svc_rbf.fit((X_train), y_train)

# Pass the test data to the predict function 
%time y_pred = svc_rbf.predict((X_test))
print(classification_report(y_test, y_pred))

### Polynomial kernel

In [ ]:
# create SVC with the optimized hyperparameters from gridsearch
svc_poly = poly_rcv.best_estimator_

# Pass the scaled train data to the SVC classifier
%time svc_poly.fit((X_train), y_train)

# Pass the test data to the predict function 
%time y_pred = svc_poly.predict((X_test))
print(classification_report(y_test, y_pred))

### Sigmoid kernel

In [ ]:
# create SVC with the optimized hyperparameters from gridsearch
svc_sig = sig_rcv.best_estimator_

# Pass the scaled train data to the SVC classifier
%time svc_sig.fit((X_train), y_train)

# Pass the test data to the predict function 
%time y_pred = svc_sig.predict((X_test))
print(classification_report(y_test, y_pred))

# Validation Curve

In [ ]:
# Varying gamma
gamma = [i for i in np.linspace(0.0001, 0.001, 10)]
fig, ax = plt.subplots(figsize=(8, 6))
val_curve_1 = ValidationCurve(SVC(C=100, kernel='rbf', probability=True),
                      param_name='gamma',
                      param_range=gamma,
                      cv=TimeSeriesSplit(n_splits=5),
                      scoring='accuracy',
                      n_jobs=-1,
                      ax=ax)
val_curve_1.fit(X_train, y_train)
val_curve_1.poof()
sns.despine()
fig.tight_layout();

In [ ]:
# Varying C
C = [1e2, 1e3, 1e4, 1e5,1e6]
fig, ax = plt.subplots(figsize=(8, 6))
val_curve_2 = ValidationCurve(SVC(kernel='rbf', gamma=0.001, probability=True),
                      param_name='C',
                      param_range=C,
                      cv=TimeSeriesSplit(n_splits=5),
                      scoring='accuracy',
                      n_jobs=2,
                      ax=ax)
val_curve_2.fit(X_train, y_train)
val_curve_2.poof()
sns.despine()
fig.tight_layout();

# Learning Curve

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
l_curve = LearningCurve(SVC(C=100, kernel='rbf', gamma=0.001, probability=True), 
                        train_sizes=np.arange(.1, 1.01, .1),
                        scoring='accuracy', 
                        cv=TimeSeriesSplit(n_splits=5), 
                        n_jobs=8,
                        ax=ax)
l_curve.fit(X_train, y_train)
l_curve.poof()
sns.despine()
fig.tight_layout();


## k-Fold cross validation (generalization error)

In [ ]:
cv_score = cross_val_score(estimator=rbf_rcv.best_estimator_,
                          X=ss.fit_transform(X_train),
                          y=y_train,
                          cv=TimeSeriesSplit(n_splits=10),
                          n_jobs=-1,
                          verbose=1)

print(cv_score)
print('-'*20)
"Accuracy: %.3f%% (%.3f%%)" % (cv_score.mean()*100.0, cv_score.std()*100.0)

In [ ]:
cv_score = cross_val_score(estimator=poly_rcv.best_estimator_,
                          X=ss.fit_transform(X_train),
                          y=y_train,
                          cv=TimeSeriesSplit(n_splits=10),
                          n_jobs=-1,
                          verbose=1)

print(cv_score)
print('-'*20)
"Accuracy: %.3f%% (%.3f%%)" % (cv_score.mean()*100.0, cv_score.std()*100.0)

## Performance metric

In [ ]:
rbf_rcv.best_estimator_

In [ ]:
kf = KFold(n_splits=10, shuffle=False)
y_score = cross_val_predict(SVC(C=100.0, gamma=0.001, probability=True),
                            X=ss.transform(X), y=y,
                           cv=kf.split(ss.transform(X)),
                            method='predict_proba')

pred_scores = dict(y_true=y, y_score=y_score)

# ROC AUC
roc_auc_score(**pred_scores, multi_class='ovr')

In [ ]:
poly_rcv.best_estimator_

In [ ]:
kf = KFold(n_splits=10, shuffle=False)
y_score = cross_val_predict(SVC(C=1000.0, gamma=0.01, kernel='poly', probability=True),
                            X=ss.transform(X), y=y,
                           cv=kf.split(ss.transform(X)),
                            method='predict_proba')

pred_scores = dict(y_true=y, y_score=y_score)

# ROC AUC
roc_auc_score(**pred_scores, multi_class='ovr')

# 5. Neutral Networks (MLP)

## Hyper-parameter tuning

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from itertools import product

In [ ]:
steps = [('scaler', StandardScaler()), ('mlp', MLPClassifier(random_state=42))]
pipeline = Pipeline(steps)
ss = StandardScaler().fit(X_train)

# Initialize parameters

parameters = {
    'mlp__hidden_layer_sizes': [(40, 30, 100), (40, 30, 40),
                                (30, 20, 30), (20, 20, 20, 10)],
    'mlp__activation': ['tanh', 'relu', 'logistic'],
    'mlp__solver': ['sgd', 'adam', 'lbfgs'],
    'mlp__alpha': [0.0001,0.001,0.01, 0.05],
    'mlp__learning_rate': ['constant','adaptive'],
    'mlp__early_stopping':[True, False] }

mlp_gcv = GridSearchCV(estimator=pipeline,
                        param_grid=parameters,
                        cv=TimeSeriesSplit(n_splits=5),
                        scoring='roc_auc',
                        n_jobs=8)

# Training on and fetching the best parameters
mlp_gcv.fit(ss.transform(X_train), y_train)

# Obtain the best parameters
best_hidden_layer_sizes = mlp_gcv.best_params_['mlp__hidden_layer_sizes']
best_activation = mlp_gcv.best_params_['mlp__activation']
best_solver = mlp_gcv.best_params_['mlp__solver']
best_alpha = mlp_gcv.best_params_['mlp__alpha']
best_learning_rate = mlp_gcv.best_params_['mlp__learning_rate']

In [ ]:
mlp_gcv.best_params_

## Validation Curve

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(40, 30, 100),activation='tanh',random_state=42,
                       solver='sgd',learning_rate='constant',alpha= 0.0001, early_stopping=False, max_iter=200)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
val_curve_2 = ValidationCurve(mlp,
                      param_name='max_iter',
                      param_range=np.arange(1,25),
                      cv=TimeSeriesSplit(n_splits=5),
                      scoring='accuracy',
                      n_jobs=-1,
                      ax=ax)
val_curve_2.fit(X_train, y_train)
val_curve_2.poof()
sns.despine()
fig.tight_layout();

# Learning curve

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
l_curve = LearningCurve(MLPClassifier(hidden_layer_sizes=(40, 30, 100),
                        activation='tanh',random_state=42,
                        solver='sgd',learning_rate='constant',
                        alpha= 0.0001, early_stopping=False, 
                        max_iter=200), 
                        train_sizes=np.arange(.1, 1.01, .1),
                        scoring='accuracy', 
                        cv=TimeSeriesSplit(n_splits=5), 
                        n_jobs=8,
                        ax=ax)
l_curve.fit(X_train, y_train)
l_curve.poof()
sns.despine()
fig.tight_layout();

# Training (loss) curve

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(40, 30, 100),activation='tanh',random_state=42,
                       solver='sgd',learning_rate='constant',alpha= 0.0001, early_stopping=False, max_iter=200)

# re-split training data into traing and validation set
t = .8
split = int(t*len(X_train))
X_tr,X_val,y_tr,y_val=X_train[:split], X_train[split:], y_train[:split], y_train[split:]


mlp.fit(X_tr, y_tr)
print(mlp.score(X_tr, y_tr))
plt.plot(mlp.loss_curve_, label='Training loss')
mlp.fit(X_val, y_val)
plt.plot(mlp.loss_curve_,label='Validation loss')
plt.xlabel('Number of iterations')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Final model prediction

In [ ]:
# create MLP with the optimized hyperparameters from gridsearch
mlp_model = MLPClassifier(hidden_layer_sizes=(40, 30, 100),
                        activation='tanh',random_state=42,
                        solver='sgd',learning_rate='constant',
                        alpha= 0.0001, early_stopping=False, 
                        max_iter=200)

ss = StandardScaler().fit(X_train)

# Pass the scaled train data to the MLP classifier
%time mlp_model.fit(ss.fit_transform(X_train), y_train)

# Pass the test data to the predict function 
%time y_pred = mlp_model.predict(ss.transform(X_test))
print(classification_report(y_test, y_pred))

## k-Fold Cross Validation (Generalization error)

In [ ]:
cv_score = cross_val_score(estimator=mlp_model,
                          X=ss.fit_transform(X_train),
                          y=y_train,
                          cv=TimeSeriesSplit(n_splits=10),
                          n_jobs=-1,
                          verbose=1)

print(cv_score)
print('-'*20)
"Accuracy: %.3f%% (%.3f%%)" % (cv_score.mean()*100.0, cv_score.std()*100.0)

## Performance metrics

In [ ]:
kf = KFold(n_splits=10, shuffle=False)
y_score = cross_val_predict(mlp_model, X=ss.transform(X), y=y,
                           cv=kf.split(ss.transform(X)),
                            method='predict_proba')

pred_scores = dict(y_true=y, y_score=y_score)

# ROC AUC
roc_auc_score(**pred_scores, multi_class='ovr')

In [ ]:
# SP500 cumulative return
df['sp500_cum_ret']=df.ret1.cumsum()

# NN performance
df['predicted_signal_nn'] = mlp_model.predict(ss.fit_transform(X))
df['strategy_returns_nn']=df.retFut1 * df.predicted_signal_nn

# SVC performance
df['predicted_signal_svc_rbf'] = svc_rbf.predict(ss.fit_transform(X))
df['strategy_returns_svc_rbf']=df.retFut1 * df.predicted_signal_svc_rbf

df['predicted_signal_svc_poly'] = svc_poly.predict(ss.fit_transform(X))
df['strategy_returns_svc_poly']=df.retFut1 * df.predicted_signal_svc_poly

# k-NN performance (k=5)
df['predicted_signal_knn_manh'] = knn_model.predict(ss.fit_transform(X))
df['strategy_returns_knn_manh']=df.retFut1 * df.predicted_signal_knn_manh

# DTree
df['predicted_signal_dt'] = model.predict(X)
df['strategy_returns_dt']=df.retFut1 * df.predicted_signal_dt

# adaBoost DTree
df['predicted_signal_ada'] = ada_boost.predict(X)
df['strategy_returns_ada']=df.retFut1 * df.predicted_signal_ada

In [ ]:
insample_period = df[:split]
outsample_period = df[split:]

In [ ]:
insample_period['sp500_cum_ret']=insample_period.ret1.cumsum()
insample_period['nn_cum_ret']=insample_period.strategy_returns_nn.cumsum()
insample_period['svc_rbf_cum_ret'] = insample_period.strategy_returns_svc_rbf.cumsum()
insample_period['svc_poly_cum_ret'] = insample_period.strategy_returns_svc_poly.cumsum()
insample_period['knn_cum_ret'] = insample_period.strategy_returns_knn_manh.cumsum()
insample_period['dt_cum_ret'] = insample_period.strategy_returns_dt.cumsum()
insample_period['ada_cum_ret'] = insample_period.strategy_returns_ada.cumsum()

outsample_period['sp500_cum_ret']=outsample_period.ret1.cumsum()
outsample_period['nn_cum_ret']=outsample_period.strategy_returns_nn.cumsum()
outsample_period['svc_rbf_cum_ret'] = outsample_period.strategy_returns_svc_rbf.cumsum()
outsample_period['svc_poly_cum_ret'] = outsample_period.strategy_returns_svc_poly.cumsum()
outsample_period['knn_cum_ret'] = outsample_period.strategy_returns_knn_manh.cumsum()
outsample_period['dt_cum_ret'] = outsample_period.strategy_returns_dt.cumsum()
outsample_period['ada_cum_ret'] = outsample_period.strategy_returns_ada.cumsum()

In [ ]:
insample_returns = insample_period.loc[:, ['Date','sp500_cum_ret','nn_cum_ret','svc_rbf_cum_ret',
                       'svc_poly_cum_ret','knn_cum_ret',
                      'dt_cum_ret','ada_cum_ret']]

insample_returns.set_index('Date', inplace=True)
insample_returns.columns =['SPY_ETF','NN','SVC_rbf','SVC_poly','KNN','DTree','DTree_adaboost']

# one plug correction for normalisation (to be used for comaprison of returns)
insample_returns.iloc[0, :] = insample_returns.iloc[0, :]+0.000001

In [ ]:
fig, axl = plt.subplots()
fig.set_size_inches(9,6)    
plt.rcParams['lines.linewidth'] = 2
axl.plot(insample_returns.index, insample_returns.SPY_ETF,'-', color='black', label='Benchmark (SP500)')
axl.plot(insample_returns.index, insample_returns.NN,'-', color='red', label='Neural Networks (MLP)')
axl.plot(insample_returns.index, insample_returns.SVC_rbf,'-', color='green', label='SVC(Gaussian kernel)')
axl.plot(insample_returns.index, insample_returns.SVC_poly,'-', color='blue', label='SVC(Polynomial kernel)')
axl.plot(insample_returns.index, insample_returns.KNN,'-', color='magenta', label='KNN(Manhattan distance)')
axl.plot(insample_returns.index, insample_returns.DTree,'-', color='cyan', label='Decision Tree')
axl.plot(insample_returns.index, insample_returns.DTree_adaboost,'-', color='yellow', label='Decision Tree (boosted)')
axl.legend(frameon=True)
axl.legend(loc='upper left')
axl.set_xlabel('Date')
axl.set_ylabel('Cumulative Returns(%)')
axl.grid(False)

In [ ]:
outsample_returns = outsample_period.loc[:, ['Date','sp500_cum_ret','nn_cum_ret','svc_rbf_cum_ret',
                       'svc_poly_cum_ret','knn_cum_ret',
                      'dt_cum_ret','ada_cum_ret']]

outsample_returns.set_index('Date', inplace=True)
outsample_returns.columns =['SPY_ETF','NN','SVC_rbf','SVC_poly','KNN','DTree','DTree_adaboost']

# one plug correction for normalisation (to be used for comaprison of returns)
outsample_returns.iloc[0, :] = outsample_returns.iloc[0, :]+0.000001

In [ ]:
fig, axl = plt.subplots()
fig.set_size_inches(10,6)    
plt.rcParams['lines.linewidth'] = 2
axl.plot(outsample_returns.index, outsample_returns.SPY_ETF,'-', color='black', label='Benchmark (SP500)')
axl.plot(outsample_returns.index, outsample_returns.NN,'-', color='red', label='Neural Networks (MLP)')
axl.plot(outsample_returns.index, outsample_returns.SVC_rbf,'-', color='green', label='SVC(Gaussian kernel)')
axl.plot(outsample_returns.index, outsample_returns.SVC_poly,'-', color='blue', label='SVC(Polynomial kernel)')
axl.plot(outsample_returns.index, outsample_returns.KNN,'-', color='magenta', label='KNN(Manhattan distance)')
axl.plot(outsample_returns.index, outsample_returns.DTree,'-', color='cyan', label='Decision Tree')
axl.plot(outsample_returns.index, outsample_returns.DTree_adaboost,'-', color='yellow', label='Decision Tree (boosted)')
axl.legend(frameon=True)
axl.legend(loc='best')
axl.set_xlabel('Date')
axl.set_ylabel('Cumulative Returns(%)')
axl.vlines(x=outsample_returns.index=)
axl.grid(False)